In [2]:
! pip install --quiet git-remote-codecommit


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import boto3
import sagemaker
import time
from time import strftime

boto_session = boto3.Session()
sagemaker_session = sagemaker.Session(boto_session=boto_session)
sm_client = boto3.client("sagemaker")
region = boto_session.region_name
bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]

prefix = "sagemaker-intelimage-dvc"

print(f"account: {account}")
print(f"bucket: {bucket}")
print(f"region: {region}")
print(f"role: {role}")

account: 294495367161
bucket: sagemaker-ap-south-1-294495367161
region: ap-south-1
role: arn:aws:iam::294495367161:role/service-role/AmazonSageMaker-ExecutionRole-20221204T133204


In [4]:
from sagemaker.pytorch.processing import PyTorchProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker import get_execution_role

In [5]:
dvc_repo_url = "codecommit::ap-south-1://sagemaker-imageclf"
dvc_branch = "processed-dataset"

In [15]:
pytorch_processor = PyTorchProcessor(
    framework_version="1.12",
    py_version="py38",
    image_uri="294495367161.dkr.ecr.ap-south-1.amazonaws.com/sagemaker:latest",
    role=get_execution_role(),
    instance_type="ml.t3.xlarge",
    instance_count=1,
    base_job_name="preprocess-intel-dataset",
    env={
        "DVC_REPO_URL": dvc_repo_url,
        "DVC_BRANCH": dvc_branch,
        "GIT_USER": "Vivek Chaudhary",
        "GIT_EMAIL": "vivek.experiotech@gmail.com",
    },
)

In [16]:
input_dataset = "s3://sagemaker-ap-south-1-294495367161/intel_imageclf.zip"

In [17]:
pytorch_processor.run(
    code="preprocess.py",
    source_dir="/root/emlov2-session-12/",
    inputs=[
        ProcessingInput(
            input_name="data",
            source=input_dataset,
            destination="/opt/ml/processing/input",
        )
    ],
)


Job Name:  preprocess-intel-dataset-2022-12-10-11-33-20-711
Inputs:  [{'InputName': 'data', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-294495367161/intel_imageclf.zip', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-294495367161/preprocess-intel-dataset-2022-12-10-11-33-20-711/source/sourcedir.tar.gz', 'LocalPath': '/opt/ml/processing/input/code/', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'entrypoint', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-294495367161/preprocess-intel-dataset-2022-12-10-11-33-20-711/source/runproc.sh', 'LocalPath': '/opt/ml/processing/input/entrypoint', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3Data